# Apply the strategy derived from LabelPropagation-exporation
* Consecutive events sharing the same tab-url are assumed to belong to the same task.
* Exception: Multiple anchor events with different labels may be included in the same segment.

## Exception handling: 
* Loglikelihood testing for consecutive events: p(i, j)/p(i)p(j) ~ Chi square p-value < 0.5
* non-exchangable: p(i|j) > p(j|i)

### Setup

In [3]:
%matplotlib inline
from pymongo import MongoClient
import pymongo
import numpy as np
import itertools
import matplotlib.pylab as pylab

# DB connections
client = MongoClient()
db = client.db_tasklog_clean
# Collections
User = db.user
Log = db.log_chrome

# users
allUsers = list(User.find({}))

In [4]:
userLogs = {}
for u in allUsers:
    user_log = Log.aggregate([
        {'$match': {'userid': u['userid'], 'removed': False}},
        {'$project': {'affected_tab_id': 1, 
                     'event': 1, 
                     'url': 1,
                     'timestamp_bson': 1, 
                     'taskname': {'$ifNull': ['$annotation.task.name', 'NA']},
                     'taskid': {'$ifNull': ['$annotation.task.taskid', 'NA']},
                     'title': {'$ifNull': ['$details.current_tab.title', 'NA']},
                     }}, 
        {'$sort':{'timestamp_bson': pymongo.ASCENDING}},
        ])
    userLogs[u['userid']] = list(user_log)
        

## First pass: group by tab-url

## Second pass:  loglikelihood test GOF